In [1]:
from bs4 import BeautifulSoup as soup
import requests
import re

In [2]:
#url for desired business
url = 'https://www.yelp.com/biz/vault-brewing-yardley'
response = requests.get(url)
text = soup(response.text,'html.parser')

In [3]:
response.status_code #used to confirm whether data can be scraped. 200 indicats that scraping will be accepted.

200

In [4]:
num_reviews = 556

In [5]:
url_list = []
for i in range(0, num_reviews,20):
    url_list.append(url+'?start='+str(i))
print(url_list[:10])

['https://www.yelp.com/biz/vault-brewing-yardley?start=0', 'https://www.yelp.com/biz/vault-brewing-yardley?start=20', 'https://www.yelp.com/biz/vault-brewing-yardley?start=40', 'https://www.yelp.com/biz/vault-brewing-yardley?start=60', 'https://www.yelp.com/biz/vault-brewing-yardley?start=80', 'https://www.yelp.com/biz/vault-brewing-yardley?start=100', 'https://www.yelp.com/biz/vault-brewing-yardley?start=120', 'https://www.yelp.com/biz/vault-brewing-yardley?start=140', 'https://www.yelp.com/biz/vault-brewing-yardley?start=160', 'https://www.yelp.com/biz/vault-brewing-yardley?start=180']


In [6]:
reviews = text.find_all('div',attrs = {'class':"lemon--div__373c0__1mboc sidebarActionsHoverTarget__373c0__2kfhE arrange__373c0__UHqhV gutter-12__373c0__3kguh grid__373c0__29zUk layout-stack-small__373c0__3cHex border-color--default__373c0__2oFDT"})

In [7]:
print(len(reviews))

20


In [8]:
review = reviews[2]
#To protect privacy, Yelp user names have not been included in the file. However, source code is below.
#username = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2pB8f fs-block text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa"}).string
#print(username)

In [9]:
location = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa text-size--small__373c0__3SGMi"}).get_text().strip()

In [10]:
print(location)

trenton, NJ


In [11]:
rating = float(review.select('[aria-label*=rating]')[0]['aria-label'].strip(' star rating'))

In [12]:
print(rating)

4.0


In [13]:
date = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2pB8f text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_"}).get_text()
print(date)

10/27/2019


In [14]:
content = review.find('p',attrs = {'class':"lemon--p__373c0__3Qnnj text__373c0__2pB8f comment__373c0__3EKjH text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_"}).text

In [15]:
content

"This restaurant/bar really reminds me of my old stomping grounds in Haddon Township NJ and especially a restaurant I used to frequent called Keg and Kitchen. This restaurant is located in an old bank building, hence the name. It definitely has an interesting interior, there is also outdoor seating and more traditional seating outside the vault area. I hear it gets pretty busy at prime bar hours, however we came here around 5 pm on a Thursday evening and there was ample seating available both indoors and out. I would think it's best to come here with a group of people as they had big platters for $17-$20 that could feed probably 4-5 people and the food looked really good. They also had a really nice selection of seasonal craft beers. We got the mushroom pizza and waffle grilled cheese. The crust of the mushroom pizza was soft on the inside and crispy on the outside, just right. Keg and Kitchen makes the same pizza, mushrooms, arugula, and truffle oil, but K&K's tastes a lot better for 

In [16]:
import csv
import time
import random

def scrape_single_page(reviews, csvwriter):
    for review in reviews:
        dic = {}
        #Code copied from previous steps
        username = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2pB8f fs-block text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa"}).string
        location = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa text-size--small__373c0__3SGMi"}).get_text().strip()
        date = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2pB8f text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_"}).get_text()
        rating = float(review.select('[aria-label*=rating]')[0]['aria-label'].strip(' star rating'))
        content = review.find('p',attrs = {'class':"lemon--p__373c0__3Qnnj text__373c0__2pB8f comment__373c0__3EKjH text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_"}).text
        #Assign Values to the dictionary
        #dic['username']=username Commented out to protect yelp user privacy
        dic['location']=location
        dic['date'] = date
        dic['rating'] = rating
        dic['content']=content
        review_writer.writerow(dic.values())
with open('vaultbrewing.csv','w',encoding = 'utf-8') as csvfile:
    review_writer = csv.writer(csvfile)
    for index, url in enumerate(url_list):
        response = requests.get(url).text
        s = soup(response, 'html.parser')
        reviews = s.find_all('div',attrs = {'class':"lemon--div__373c0__1mboc sidebarActionsHoverTarget__373c0__2kfhE arrange__373c0__UHqhV gutter-12__373c0__3kguh grid__373c0__29zUk layout-stack-small__373c0__3cHex border-color--default__373c0__2oFDT"})
        scrape_single_page(reviews, review_writer)
        #Random sleep to avoid getting banned from the server
        time.sleep(random.randint(1,5))
        #log the progress
        print('finished page '+str(index+1) + ' of '+ str(len(url_list)))

finished page 1 of 28
finished page 2 of 28
finished page 3 of 28
finished page 4 of 28
finished page 5 of 28
finished page 6 of 28
finished page 7 of 28
finished page 8 of 28
finished page 9 of 28
finished page 10 of 28
finished page 11 of 28
finished page 12 of 28
finished page 13 of 28
finished page 14 of 28
finished page 15 of 28
finished page 16 of 28
finished page 17 of 28
finished page 18 of 28
finished page 19 of 28
finished page 20 of 28
finished page 21 of 28
finished page 22 of 28
finished page 23 of 28
finished page 24 of 28
finished page 25 of 28
finished page 26 of 28
finished page 27 of 28
finished page 28 of 28
